**Class Definition and Initialization**

This section defines the class `TspDynamicProgrammingIterative` and initializes its fields. It includes validation checks to ensure the input matrix is square and the starting node is within valid bounds.

In [9]:
from itertools import combinations

class TspDynamicProgrammingIterative:
    def __init__(self, distance, start=0):
        self.distance = distance
        self.N = len(distance)
        self.start = start
        self.minTourCost = float('inf')
        self.tour = []
        self.ranSolver = False

        if self.N <= 2:
            raise ValueError("N <= 2 not supported.")
        if self.N != len(distance[0]):
            raise ValueError("Matrix must be square (N x N)")
        if self.start < 0 or self.start >= self.N:
            raise ValueError("Invalid start node.")
        if self.N > 32:
            raise ValueError("Matrix too large!")

    def not_in(self, elem, subset):
        return ((1 << elem) & subset) == 0

    def get_tour(self):
        if not self.ranSolver:
            self.solve()
        return self.tour

    def get_tour_cost(self):
        if not self.ranSolver:
            self.solve()
        return self.minTourCost

    def combinations(self, r, n):
        result = []
        for subset in combinations(range(n), r):
            bits = 0
            for bit in subset:
                bits |= 1 << bit
            result.append(bits)
        return result

    def solve(self):
        if self.ranSolver:
            return

        END_STATE = (1 << self.N) - 1
        memo = [[float('inf')] * (1 << self.N) for _ in range(self.N)]

        for end in range(self.N):
            if end == self.start:
                continue
            memo[end][(1 << self.start) | (1 << end)] = self.distance[self.start][end]

        for r in range(3, self.N + 1):
            for subset in self.combinations(r, self.N):
                if self.not_in(self.start, subset):
                    continue
                for next in range(self.N):
                    if next == self.start or self.not_in(next, subset):
                        continue

                    subset_without_next = subset ^ (1 << next)
                    min_dist = float('inf')
                    for end in range(self.N):
                        if end == self.start or end == next or self.not_in(end, subset):
                            continue
                        prev_dist = memo[end][subset_without_next] + self.distance[end][next]
                        if prev_dist < min_dist:
                            min_dist = prev_dist

                    memo[next][subset] = min_dist

        for i in range(self.N):
            if i == self.start:
                continue
            tour_cost = memo[i][END_STATE] + self.distance[i][self.start]
            if tour_cost < self.minTourCost:
                self.minTourCost = tour_cost

        last_index = self.start
        state = END_STATE
        self.tour.append(self.start)

        for _ in range(self.N - 1):
            best_index = -1
            best_dist = float('inf')
            for j in range(self.N):
                if j == self.start or self.not_in(j, state):
                    continue
                new_dist = memo[j][state] + self.distance[j][last_index]
                if new_dist < best_dist:
                    best_index = j
                    best_dist = new_dist
            self.tour.append(best_index)
            state ^= (1 << best_index)
            last_index = best_index

        self.tour.append(self.start)
        self.tour.reverse()

        self.ranSolver = True


**Methods to Get Tour and Tour Cost**

These methods return the optimal tour and its cost. If the solver hasn't been run yet, they call the `solve()` method.

**Helper Methods**

These helper methods generate combinations of subsets and check if an element is in a subset.

**Solve Method**

The `solve()` method initializes the state and memoization structures, iterates through subsets of nodes to compute the minimum tour cost, and reconstructs the path based on the memo table.

**Example scenario**

In [13]:
!pip install pympler

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.8/165.8 kB 6.5 MB/s eta 0:00:00


In [14]:
import time
import pandas as pd
import tracemalloc
from pympler import asizeof

def generate_symmetric_distance_matrix(n, seed=None, min_val=1, max_val=100):
    import random
    if seed is not None:
        random.seed(seed)
    matrix = [[0]*n for _ in range(n)]
    for i in range(n):
        for j in range(i):
            val = random.randint(min_val, max_val)
            matrix[i][j] = matrix[j][i] = val
    return matrix

def test_iterative_tsp(sizes, seed_base=123):
    results = []

    for size in sizes:
        try:
            dist = generate_symmetric_distance_matrix(size, seed=seed_base + size)
            tsp = TspDynamicProgrammingIterative(dist)
            tracemalloc.start()
            start = time.time()
            tsp.solve()
            duration = time.time() - start
            current, peak = tracemalloc.get_traced_memory()
            tracemalloc.stop()
            results.append({
                "approach": "Iterative",
                "size": size,
                "cost": tsp.get_tour_cost(),
                "tour": tsp.get_tour(),
                "time": duration,
                "mem_peak_kb": peak / 1024,
                "mem_object_kb": asizeof.asizeof(tsp) / 1024
            })
        except Exception as e:
            results.append({
                "approach": "Iterative",
                "size": size,
                "cost": None,
                "tour": None,
                "time": None,
                "error": str(e)
            })

    return pd.DataFrame(results)


In [15]:
df = test_iterative_tsp([4, 5, 6, 7, 8, 9, 10, 11, 12, 13])  # or test_iterative_tsp(...)
df[["approach", "size", "cost", "time", "mem_peak_kb", "mem_object_kb"]]


,approach,size,cost,time,mem_peak_kb,mem_object_kb
0,Iterative,4,103,0.000219,0.976562,1.507812
1,Iterative,5,156,0.000318,1.710938,1.812500
2,Iterative,6,155,0.000772,3.593750,2.132812
3,Iterative,7,269,0.002686,8.678711,2.437500
4,Iterative,8,203,0.006160,17.718750,2.945312
5,Iterative,9,216,0.012395,44.303711,3.156250
6,Iterative,10,223,0.028595,90.367188,3.664062
7,Iterative,11,269,0.104832,210.039062,3.968750
8,Iterative,12,223,0.187589,471.733398,4.476562
9,Iterative,13,168,0.489231,911.739258,4.882812
